In the new architecture of the DiffGFDN, we have decoupled FDNs, each having the common decay times associated with each room in the coupled space, The block diagonal mixing matrix, $\mathbf{A}$ and the input and output gains, $\mathbf{b,c}$ are determined using colourless FDN optimisation. The only thing left to be determined is the source and receiver filters, $\mathbf{g_i}(z), \mathbf{g_o}(z)$. Using the common-slopes model, we see that these gains model the position-dependent amplitudes associated with each common slope. Can we find these filters by using a rank-1 decomposition of the common slope amplitudes in each frequency band?

Let's first start by investigating the broadband case, where the source-receiver filters are replaced with broadband gains. We test our hypothesis on a synthetically generated dataset of shaped white noise, which has two slopes, each with decay time $0.5$s and $1.7$s. The amplitudes are spatial functions of the room geometry and source-receiver positions. Let the amplitude of the $k$th slope at source position $\mathbf{x_s}$ and receiver position $\mathbf{x_r}$ be denoted as $A_k(\mathbf{x_s, x_r}).$ On doing a common-slope analysis of RIRs at source positions $\mathbf{x_{s_1}} \cdots, \mathbf{x_{s_M}}$ and receiver multiple receiver positions $\mathbf{x_{r_1}}, \cdots, \mathbf{x_{r_N}}$, we can get a matrix of amplitudes for the $k$th slope, 

\begin{aligned}
\mathbf{A}_k = \begin{bmatrix}A_k(\mathbf{x_{s_1}, x_{r_1}}) & A_k(\mathbf{x_{s_1}, x_{r_2}}) & \cdots  & A_k(\mathbf{x_{s_1}, x_{r_N}}) \\
\vdots & \vdots & \ddots & \vdots \\
A_k(\mathbf{x_{s_M}, x_{r_1}}) & A_k(\mathbf{x_{s_M}, x_{r_2}}) & \cdots  & A_k(\mathbf{x_{s_M}, x_{r_N}})
\end{bmatrix}
\end{aligned}

We can do a rank-1 decomposition of this matrix using singular value decomposition
\begin{aligned}
\mathbf{A}_k  &= \mathbf{U_k} \Sigma_k \mathbf{V_k}^{-1} \\
& \approx \sigma_{1k} \mathbf{u}_{1k} \mathbf{v}_{1k}^{H}
\end{aligned}
where $\mathbf{u}_{1k} \in \mathbb{C}^{M \times 1}, \mathbf{v}_{1k} \in \mathbb{C}^{N \times 1}$ are the left and right singular vectors associated with the largest singular value, $\sigma_{1k}$. Using this decomposition, we can set the source and receiver gains for the $k$th FDN as, 

\begin{aligned}
\mathbf{g_{i_k}} &= \begin{bmatrix} g_{i_k}(\mathbf{x_{s_1}}) & \cdots & g_{i_k}(\mathbf{x_{s_M}})\end{bmatrix}^T = \sqrt{\sigma_{1k}}\mathbf{u}_{1k} \\
\mathbf{g_{o_k}} &= \begin{bmatrix} g_{o_k}(\mathbf{x_{r_1}}) & \cdots & g_{o_k}(\mathbf{x_{r_N}})\end{bmatrix}^T = \sqrt{\sigma_{1k}}\mathbf{v}_{1k}
\end{aligned}

We can pre-compute these gains from common-slope analysis and interpolate between them for dynamic source and listener positions. Now, if we do not have a regular grid of source-receiver positions, for example, for source location $\mathbf{x_{s_m}}$, we only have measured RIRs at $P < N$ receiver positions, then some entries in the matrix $\mathbf{A}_k$ will be 0. We can still recover the source and receiver gains by minimising the loss:

\begin{aligned}
\mathcal{L}(\mathbf{g}_{ik}, \mathbf{g}_{ok}) := ||\mathbf{g}_{ik} \mathbf{g}_{ok}^T - \mathbf{A}_k||_F
\end{aligned}
We can initialise $\mathbf{g}_{ik}, \mathbf{g}_{ok}$ with the rank-1 decomposition suggested above, and then use gradient descent to converge to an optimal solution of $\mathbf{g}_{ik}^*, \mathbf{g}_{ok}^*$
To fix scaling ambiguities, i.e., the optimum value of the source gains is found as, $\mathbf{g}^*_{ik}\alpha$ and the optimum values of receiver gains is found as, $\frac{\mathbf{g}^*_{ok}}{\alpha}$ for some non-zero $\alpha$, a modified loss is considered,

\begin{aligned}
\mathcal{L}^* (\mathbf{g}_{ik}, \mathbf{g}_{ok}) := ||\mathbf{g}_{ik} \mathbf{g}_{ok}^T - \mathbf{A}_k||_F + \lambda (\mathbf{g}_{ik}^T \mathbf{g}_{ik} - \mathbf{g}_{ok}^T\mathbf{g}_{ok})
\end{aligned}